## Import the data

Create the datapoint

In [1]:
import base64
import json
from io import BytesIO
from pathlib import Path
from typing import Any, Dict, Generator, List, Optional

from datasets import Dataset, Features
from datasets import Image as DSImage
from datasets import Sequence, Value, load_dataset
from PIL import Image
from tqdm import tqdm

In [2]:
ds = load_dataset("osunlp/Multimodal-Mind2Web")

Resolving data files:   0%|          | 0/27 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/23 [00:00<?, ?it/s]

In [3]:
# Declare classes
import dataclasses
from dataclasses import dataclass
from typing import List, Literal, Tuple
import json


@dataclass
class Coordinate:
    x: int
    y: int


@dataclass
class ScrollBar:
    offset: float
    height: float


@dataclass
class BrowserState:
    screenshot: str
    height: int
    width: int
    scrollbar: ScrollBar
    url: str
    mouse: Coordinate


@dataclass
class BrowserAction:
    action: Literal[
        "success",
        "failure",
        "key",
        "type",
        "mouse_move",
        "left_click",
        "left_click_drag",
        "right_click",
        "middle_click",
        "double_click",
        "screenshot",
        "cursor_position",
        "scroll_up",
        "scroll_down",
    ]
    # TODO: Do we want to use Coordinate class here, or easier to just construct with tuple
    coordinate: tuple[int, int] | None
    text: str | None
    reasoning: str
    id: str


@dataclass
class BrowserStep:
    state: BrowserState
    action: BrowserAction

In [4]:
# Declare functions

import random


def generate_tool_id() -> str:
    prefix = "toolu_01"
    characters = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789"
    id_length = 22
    result = prefix

    for _ in range(id_length):
        result += random.choice(characters)

    return result


def is_in_viewport(viewport, point):
    x1, y1, x2, y2 = viewport
    x, y = point
    return x1 <= x <= x2 and y1 <= y <= y2


def scroll_viewport(direction, viewport, y_max):
    x1, y1, x2, y2 = viewport
    height = y2 - y1
    scroll_amount = 0.75 * height

    if direction == "up":
        new_y1 = max(1, y1 - scroll_amount)
        new_y2 = new_y1 + height
    elif direction == "down":
        new_y2 = min(y_max, y2 + scroll_amount)
        new_y1 = new_y2 - height
    else:
        raise ValueError("Direction must be 'up' or 'down'")

    # Adjust if the new viewport exceeds bounds while preserving height
    if new_y1 < 1:
        new_y1 = 1
        new_y2 = new_y1 + height
    if new_y2 > y_max:
        new_y2 = y_max
        new_y1 = new_y2 - height

    return (x1, new_y1, x2, new_y2)


def viewport_screenshot(screenshot, viewport):
    import base64
    from io import BytesIO

    x1, y1, x2, y2 = map(int, viewport)
    cropped_image = screenshot.copy().crop((x1, y1, x2, y2))

    buffered = BytesIO()
    cropped_image.save(buffered, format="JPEG", quality=85)
    encoded_string = base64.b64encode(buffered.getvalue()).decode("utf-8")

    return encoded_string
    # return ""


def process_step(
    step, mouse_coordinates: Coordinate
) -> Tuple[List[BrowserStep], Coordinate]:
    cerebellum_steps: List[BrowserStep] = []

    # Initialize the viewport to the top 16:10 ratio part of the screenshot
    screenshot = step["screenshot"]
    width, height = screenshot.size
    viewport_height = width * 10 / 16
    viewport = (0, 0, width, viewport_height)

    # Find the bounding box of the first pos_candidates
    if len(step["pos_candidates"]) == 0:
        return ([], mouse_coordinates)

    candidate = json.loads(step["pos_candidates"][0])
    attributes = json.loads(candidate["attributes"])
    bounding_box_rect = attributes["bounding_box_rect"]
    x, y, box_width, box_height = map(float, bounding_box_rect.split(","))
    center_x = x + box_width / 2
    center_y = y + box_height / 2

    if not (0 <= center_x <= width and 0 <= center_y <= height):
        print("Bounding box coordinates outside of provided screenshot, skipping step")
        return ([], mouse_coordinates)

    # Scroll the viewport until the center of the bounding box is in view
    y_max = float(height)
    while not is_in_viewport(viewport, (center_x, center_y)):
        if center_y < viewport[1]:
            browser_state = BrowserState(
                url="",
                screenshot=viewport_screenshot(screenshot, viewport),
                height=viewport_height,
                width=width,
                scrollbar=ScrollBar(
                    offset=float(viewport[1]) / y_max,
                    height=float(viewport_height) / y_max,
                ),
                mouse=mouse_coordinates,
            )
            page_up_action = BrowserAction(
                action="key",
                coordinate=None,
                text="PAGE_UP",
                reasoning="Press the Page Up key to scroll up",
                id=generate_tool_id(),
            )
            cerebellum_steps.append(
                BrowserStep(state=browser_state, action=page_up_action)
            )

            viewport = scroll_viewport("up", viewport, y_max)
        elif center_y > viewport[3]:

            browser_state = BrowserState(
                url="",
                screenshot=viewport_screenshot(screenshot, viewport),
                height=viewport_height,
                width=width,
                scrollbar=ScrollBar(
                    offset=float(viewport[1]) / y_max,
                    height=float(viewport_height) / y_max,
                ),
                mouse=mouse_coordinates,
            )
            page_down_action = BrowserAction(
                action="key",
                coordinate=None,
                text="PAGE_DOWN",
                reasoning="Press the Page Down key to scroll down",
                id=generate_tool_id(),
            )
            cerebellum_steps.append(
                BrowserStep(state=browser_state, action=page_down_action)
            )
            viewport = scroll_viewport("down", viewport, y_max)

    # Create a mouse movement action to position the mouse into the center of the bounding box
    # Remap center_x and center_y relative to the current viewport
    center_x_relative = center_x - viewport[0]
    center_y_relative = center_y - viewport[1]
    mouse_move_action = BrowserAction(
        action="mouse_move",
        coordinate=(center_x_relative, center_y_relative),
        text=None,
        reasoning="Move mouse to the center of the element",
        id=generate_tool_id(),
    )
    browser_state = BrowserState(
        url="",
        screenshot=viewport_screenshot(screenshot, viewport),
        height=viewport_height,
        width=width,
        scrollbar=ScrollBar(
            offset=float(viewport[1]) / y_max, height=float(viewport_height) / y_max
        ),
        mouse=mouse_coordinates,
    )
    move_step = BrowserStep(state=browser_state, action=mouse_move_action)
    cerebellum_steps.append(move_step)

    # Pretend now the mouse was moved
    mouse_coordinates = Coordinate(x=center_x_relative, y=center_y_relative)

    # Perform a left click action
    left_click_action = BrowserAction(
        action="left_click",
        coordinate=None,
        text=None,
        reasoning="Perform a left click on element",
        id=generate_tool_id(),
    )
    browser_state = BrowserState(
        url="",
        screenshot=viewport_screenshot(screenshot, viewport),
        height=viewport_height,
        width=width,
        scrollbar=ScrollBar(
            offset=float(viewport[1]) / y_max, height=float(viewport_height) / y_max
        ),
        mouse=mouse_coordinates,
    )
    left_click_step = BrowserStep(state=browser_state, action=left_click_action)
    cerebellum_steps.append(left_click_step)

    # Create corresponding key actions if the action is "type" or "select"
    operation = json.loads(step["operation"])
    if operation["op"] in ["TYPE", "SELECT"]:
        text = operation["value"]
        type_action = BrowserAction(
            action="type",
            coordinate=None,
            text=text,
            reasoning=f"Typing text set to desired value",
            id=generate_tool_id(),
        )
        browser_state = BrowserState(
            url="",
            screenshot=viewport_screenshot(screenshot, viewport),
            height=viewport_height,
            width=width,
            scrollbar=ScrollBar(
                offset=float(viewport[1]) / y_max, height=float(viewport_height) / y_max
            ),
            mouse=mouse_coordinates,
        )
        type_step = BrowserStep(state=browser_state, action=type_action)
        cerebellum_steps.append(type_step)

    # Return an array of BrowserStep[]
    return (cerebellum_steps, mouse_coordinates)

In [5]:
train = ds.get("train")

train_iterator = iter(train)

print(list(train[0].keys()))

data_point = next(train_iterator)
i = 1
while train_iterator is not None:
    goal = data_point["confirmed_task"]
    task_id = data_point["annotation_id"]
    action_id = data_point["action_uid"]

    print("Grabbing steps for:", goal, task_id)

    steps = [data_point]

    # Keep on pulling on the iterator until we get all the steps in this task
    while True:
        try:
            data_point = next(train_iterator)
        except StopIteration:
            train_iterator = None
            break

        if data_point["confirmed_task"] != goal:
            break

        steps.append(data_point)

    cerebellum_steps: List[BrowserStep] = []

    mouse = Coordinate(x=1, y=1)
    for raw_step in steps:

        decomposed_steps, mouse = process_step(raw_step, mouse)

        cerebellum_steps += decomposed_steps

    # Define the output file path
    output_file_path = f"mind2web/{task_id}.jsonl"

    # Open the file in write mode
    with open(output_file_path, "w") as outfile:
        goal_json = json.dumps({"goal": goal})
        outfile.write(goal_json)
        outfile.write("\n")
        # Iterate over each step in cerebellum_steps
        for this_step in cerebellum_steps:
            # Write the dictionary as a JSON line
            step_str = json.dumps(dataclasses.asdict(this_step))
            outfile.write(step_str)
            outfile.write("\n")

['action_uid', 'raw_html', 'cleaned_html', 'operation', 'pos_candidates', 'neg_candidates', 'website', 'domain', 'subdomain', 'annotation_id', 'confirmed_task', 'screenshot', 'action_reprs', 'target_action_index', 'target_action_reprs']
Grabbing steps for: rent a car in Brooklyn - Central, NY on from April 9 to April 15. 401c4e6f-6b0b-47b4-8157-92d7ca468bbc
Grabbing steps for: Show computer game reviews sorted by score. 00e83fae-f2f0-48ab-912c-32c03982642b
Grabbing steps for: Find the address and store hours for the Armageddon Shop record store in Boston. 74f01011-9bcd-433b-8405-975ca5c3f356
Grabbing steps for: Buy a copy of the Gorillaz first studio album. bd32af6e-9fae-4968-b3e6-d143f148f5b8
Grabbing steps for: Buy a pop rock album CD from the United Kingdom that was released in 2016, is between £15 and £20 and in perfect condition. c731e6d1-7194-4d16-a6a8-f5fc9be56a55
Grabbing steps for: Find a full-time job in Budget USA in finance in any location, and apply to the latest job. 6a9a

/opt/homebrew/Caskroom/miniconda/base/envs/cerebellum-train/lib/python3.11/site-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (108679428 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Grabbing steps for: Compare Pro Plans with other plans available. d4f9c67f-00c5-41ed-bde1-c704b92647f7
Grabbing steps for: Find popular tracks from the rock band Nirvana. efe9051b-d626-443c-b8c8-e7844a0acd29
Grabbing steps for: Find parking near Venice Beach that allows mobile coupons. aa1a4414-f141-42b8-9e8b-28d5bd1eeef2
Grabbing steps for: Find parking near Disneyland that has EV charging. b1fa9bb3-6e2c-485b-90f1-2a54510bf358
Grabbing steps for: Find out what payment methods are available for monthly parking. b9f5dd60-690d-4f32-9e69-3db9d346f020
Grabbing steps for: Find parking spaces near Warner Theatre in Washington DC. cdb6b70d-6083-49b5-908c-a6c0f711c175
Grabbing steps for: Add a parking button code for my company Boston Legal located near Boston Navy Yard, and the button should be 160x400 pixels, and the parking spot should be within 15 miles of my address. fbefeb82-f3e6-4db4-acc6-a68b1d519ba7
Grabbing steps for: Find the cheapest event to attend in Athens, Greece on April 2nd 7

In [6]:
import base64

CURSOR_64 = "iVBORw0KGgoAAAANSUhEUgAAAAoAAAAQCAYAAAAvf+5AAAAAw3pUWHRSYXcgcHJvZmlsZSB0eXBlIGV4aWYAAHjabVBRDsMgCP33FDuC8ijF49i1S3aDHX9YcLFLX+ITeOSJpOPzfqVHBxVOvKwqVSQbuHKlZoFmRzu5ZD55rvX8Uk9Dz2Ql2A1PVaJ/1MvPwK9m0TIZ6TOE7SpUDn/9M4qH0CciC/YwqmEEcqGEQYsvSNV1/sJ25CvUTxqBjzGJU86rbW9f7B0QHSjIxoD6AOiHE1oXjAlqjQVyxmTMkJjEFnK3p4H0BSRiWUv/cuYLAAABhWlDQ1BJQ0MgcHJvZmlsZQAAeJx9kT1Iw0AYht+2SqVUHCwo0iFD1cWCqIijVqEIFUKt0KqDyaV/0KQhSXFxFFwLDv4sVh1cnHV1cBUEwR8QZwcnRRcp8buk0CLGg7t7eO97X+6+A/yNClPNrnFA1SwjnUwI2dyqEHxFCFEM0DoqMVOfE8UUPMfXPXx8v4vzLO+6P0evkjcZ4BOIZ5luWMQbxNObls55nzjCSpJCfE48ZtAFiR+5Lrv8xrnosJ9nRoxMep44QiwUO1juYFYyVOIp4piiapTvz7qscN7irFZqrHVP/sJwXltZ5jrNKJJYxBJECJBRQxkVWIjTrpFiIk3nCQ//kOMXySWTqwxGjgVUoUJy/OB/8Lu3ZmFywk0KJ4DuF9v+GAaCu0Czbtvfx7bdPAECz8CV1vZXG8DMJ+n1thY7Avq2gYvrtibvAZc7wOCTLhmSIwVo+gsF4P2MvikH9N8CoTW3b61znD4AGepV6gY4OARGipS97vHuns6+/VvT6t8Ph1lyr0hzlCAAAA14aVRYdFhNTDpjb20uYWRvYmUueG1wAAAAAAA8P3hwYWNrZXQgYmVnaW49Iu+7vyIgaWQ9Ilc1TTBNcENlaGlIenJlU3pOVGN6a2M5ZCI/Pgo8eDp4bXBtZXRhIHhtbG5zOng9ImFkb2JlOm5zOm1ldGEvIiB4OnhtcHRrPSJYTVAgQ29yZSA0LjQuMC1FeGl2MiI+CiA8cmRmOlJERiB4bWxuczpyZGY9Imh0dHA6Ly93d3cudzMub3JnLzE5OTkvMDIvMjItcmRmLXN5bnRheC1ucyMiPgogIDxyZGY6RGVzY3JpcHRpb24gcmRmOmFib3V0PSIiCiAgICB4bWxuczp4bXBNTT0iaHR0cDovL25zLmFkb2JlLmNvbS94YXAvMS4wL21tLyIKICAgIHhtbG5zOnN0RXZ0PSJodHRwOi8vbnMuYWRvYmUuY29tL3hhcC8xLjAvc1R5cGUvUmVzb3VyY2VFdmVudCMiCiAgICB4bWxuczpkYz0iaHR0cDovL3B1cmwub3JnL2RjL2VsZW1lbnRzLzEuMS8iCiAgICB4bWxuczpHSU1QPSJodHRwOi8vd3d3LmdpbXAub3JnL3htcC8iCiAgICB4bWxuczp0aWZmPSJodHRwOi8vbnMuYWRvYmUuY29tL3RpZmYvMS4wLyIKICAgIHhtbG5zOnhtcD0iaHR0cDovL25zLmFkb2JlLmNvbS94YXAvMS4wLyIKICAgeG1wTU06RG9jdW1lbnRJRD0iZ2ltcDpkb2NpZDpnaW1wOjFiYzFkZjE3LWM5YmMtNGYzZi1hMmEzLTlmODkyNWNiZjY4OSIKICAgeG1wTU06SW5zdGFuY2VJRD0ieG1wLmlpZDo4YTUyMWJhMC00YmNlLTQzZWEtYjgyYS04ZGM2MTBjYmZlOTgiCiAgIHhtcE1NOk9yaWdpbmFsRG9jdW1lbnRJRD0ieG1wLmRpZDplODQ3ZjUxNC00MWVlLTQ2ZjYtOTllNC1kNjI3MjMxMjhlZTIiCiAgIGRjOkZvcm1hdD0iaW1hZ2UvcG5nIgogICBHSU1QOkFQST0iMi4wIgogICBHSU1QOlBsYXRmb3JtPSJMaW51eCIKICAgR0lNUDpUaW1lU3RhbXA9IjE3MzAxNTc3NjY5MTI3ODciCiAgIEdJTVA6VmVyc2lvbj0iMi4xMC4zOCIKICAgdGlmZjpPcmllbnRhdGlvbj0iMSIKICAgeG1wOkNyZWF0b3JUb29sPSJHSU1QIDIuMTAiCiAgIHhtcDpNZXRhZGF0YURhdGU9IjIwMjQ6MTA6MjhUMTY6MjI6NDYtMDc6MDAiCiAgIHhtcDpNb2RpZnlEYXRlPSIyMDI0OjEwOjI4VDE2OjIyOjQ2LTA3OjAwIj4KICAgPHhtcE1NOkhpc3Rvcnk+CiAgICA8cmRmOlNlcT4KICAgICA8cmRmOmxpCiAgICAgIHN0RXZ0OmFjdGlvbj0ic2F2ZWQiCiAgICAgIHN0RXZ0OmNoYW5nZWQ9Ii8iCiAgICAgIHN0RXZ0Omluc3RhbmNlSUQ9InhtcC5paWQ6ZTVjOTM2ZDYtYjMzYi00NzM4LTlhNWUtYjM3YTA5MzdjZDAxIgogICAgICBzdEV2dDpzb2Z0d2FyZUFnZW50PSJHaW1wIDIuMTAgKExpbnV4KSIKICAgICAgc3RFdnQ6d2hlbj0iMjAyNC0xMC0yOFQxNjoyMjo0Ni0wNzowMCIvPgogICAgPC9yZGY6U2VxPgogICA8L3htcE1NOkhpc3Rvcnk+CiAgPC9yZGY6RGVzY3JpcHRpb24+CiA8L3JkZjpSREY+CjwveDp4bXBtZXRhPgogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAKICAgICAgICAgICAgICAgICAgICAgICAgICAgCjw/eHBhY2tldCBlbmQ9InciPz5/5aQ8AAAABmJLR0QAcgByAAAtJLTuAAAACXBIWXMAAABZAAAAWQGqnamGAAAAB3RJTUUH6AocFxYuv5vOJAAAAHhJREFUKM+NzzEOQXEMB+DPYDY5iEVMIpzDfRxC3mZyBK7gChZnELGohaR58f7a7dd8bVq4YaVQgTvWFVjCUcXxA28qcBBHFUcVRwWPPuFfXVsbt0PPnLBL+dKHL+wxxhSPhBcZznuDXYKH1uGzBJ+YtPAZRyy/jTd7qEoydWUQ7QAAAABJRU5ErkJggg=="
CURSOR_BYTES = base64.b64decode(CURSOR_64)

In [7]:
# Post process
system_prompt = """You are an intelligent web browsing agent operating in fullscreen mode to accomplish a specified user goal, detailed in <USER_TASK>.
* Use only the Page Down or Page Up keys for scrolling.
* If the webpage is scrollable, a gray rectangular scrollbar will appear on the right edge of the screenshot.
* Adhere strictly to the instructions in the <IMPORTANT> section below.
</SYSTEM_CAPABILITY>

Your task is to execute user requests using their browser. After each action, capture a screenshot and thoroughly assess whether the desired outcome has been achieved. Clearly articulate your reasoning for each function call: "I have evaluated step X..." If the result is incorrect, attempt the step again. Proceed to the next step only after confirming successful execution. Always utilize a tool for actions and ensure to return a tool call. Remember to invoke the stop_browsing tool upon achieving the task's goal. Prioritize keyboard shortcuts for navigation whenever feasible.

<IMPORTANT>
* Utilize the user's <USER_DATA> to complete forms as you progress towards the goal.
* Ensure a UI element is fully visible before interacting with it.
</IMPORTANT>"""


import io
import json
import math
import os

from PIL import Image

cursor_img = Image.open(io.BytesIO(CURSOR_BYTES))


def mark_screenshot(
    img_buffer: bytes, mouse_position: Coordinate, scrollbar: ScrollBar
) -> str:
    """Adds scrollbar and cursor overlays to a screenshot.
    Args:
        img_buffer: Raw bytes of the screenshot image
        mouse_position: Coordinate object containing x,y position of mouse cursor
        scrollbar: ScrollBar object containing scrollbar dimensions and position
    Returns:
        Base64 string of the modified screenshot with overlays added
    Raises:
        IOError: If there are issues manipulating the image
    """
    with Image.open(io.BytesIO(img_buffer)) as img:
        width, height = img.size

        # Create scrollbar overlay
        scrollbar_width = 10
        scrollbar_height = int(height * scrollbar.height)
        scrollbar_top = int(height * scrollbar.offset)

        # Create gray rectangle for scrollbar with 70% opacity
        scrollbar_img = Image.new(
            "RGBA", (scrollbar_width, scrollbar_height), (128, 128, 128, int(255 * 0.8))
        )

        # Create composite image
        composite = img.copy()
        composite.paste(
            scrollbar_img, (width - scrollbar_width, scrollbar_top), scrollbar_img
        )

        # Add cursor
        composite.paste(
            cursor_img,
            (max(0, mouse_position.x), max(0, mouse_position.y)),
            cursor_img,
        )

        # Calculate the aspect ratio
        aspect_ratio = composite.width / composite.height

        # Determine the new dimensions while preserving aspect ratio
        if composite.width > 640 or composite.height > 400:
            if aspect_ratio > 640 / 400:
                new_width = 640
                new_height = int(640 / aspect_ratio)
            else:
                new_height = 400
                new_width = int(400 * aspect_ratio)
        else:
            new_width, new_height = composite.width, composite.height

        # Resize the composite image
        composite = composite.resize((new_width, new_height), Image.LANCZOS)

        # Convert back to base64 string
        output_buffer = io.BytesIO()
        composite.save(output_buffer, "JPEG", quality=85)
        return base64.b64encode(output_buffer.getvalue()).decode("utf-8")


# Define the directory containing the jsonl files
directory = "mind2web"

# Loop through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".jsonl"):
        file_path = os.path.join(directory, filename)

        print("Processing", filename)
        # Open and read each jsonl file
        with open(file_path, "r") as file:
            lines = []
            for line in file:
                # Process each line as a JSON object
                json_object = json.loads(line)
                lines.append(json_object)

            goal = lines.pop(0)["goal"]

            starting_directions = []

            starting_directions.append({"role": "system", "content": system_prompt})
            starting_directions.append(
                {
                    "role": "user",
                    "content": f"<USER_TASK>{goal}</USER_TASK>\n<USER_DATA>NONE</USER_DATA>",
                }
            )

            screenshot_tool_use_id = generate_tool_id()

            # Play with this for best behavior
            starting_directions.append(
                {
                    "role": "assistant",
                    "content": [
                        {
                            "type": "text",
                            "text": "Take a screenshot of the browser to understand the current webpage",
                        },
                        {
                            "type": "tool_use",
                            "id": screenshot_tool_use_id,
                            "name": "computer",
                            "input": {"action": "screenshot"},
                        },
                    ],
                }
            )

            examples = []

            # lines now contains the steps
            for i in range(len(lines)):

                # Need to add some logic to track last tool use id
                last_function_name = "screenshot"
                last_function_use_id = screenshot_tool_use_id

                subset = lines[: i + 1]

                training_example = starting_directions.copy()

                for step_idx in range(len(subset)):
                    step = subset[step_idx]
                    state = step["state"]

                    scrollbar = ScrollBar(
                        offset=step["state"]["scrollbar"]["offset"],
                        height=step["state"]["scrollbar"]["height"],
                    )
                    mouse = Coordinate(
                        x=math.floor(step["state"]["mouse"]["x"]),
                        y=math.floor(step["state"]["mouse"]["y"]),
                    )

                    normalized_mouse_x = mouse.x / float(step["state"]["width"])
                    normalized_mouse_y = mouse.y / float(step["state"]["height"])

                    tool_result_content = {
                        "type": "tool_result",
                        "tool_use_id": last_function_use_id,
                        "is_error": False,
                        "content": [],
                    }

                    tool_result_content["content"].append(
                        {
                            "type": "text",
                            "value": f"Mouse is at X: {normalized_mouse_x}, Y: {normalized_mouse_y}",
                        }
                    )

                    if step_idx == len(subset) - 1:
                        decoded_img = base64.b64decode(step["state"]["screenshot"])
                        marked_image = mark_screenshot(decoded_img, mouse, scrollbar)
                        tool_result_content["content"].append(
                            {
                                "type": "image",
                                "source": {
                                    "type": "base64",
                                    "media_type": "image/jpeg",
                                    "data": marked_image,
                                },
                            }
                        )

                    training_example.append(
                        {"role": "user", "content": [tool_result_content]}
                    )

                    # Build tool_use with reasoning text block and tool_use block in content
                    tool_use_content = [
                        {"type": "text", "text": step["action"]["reasoning"]}
                    ]

                    current_tool_use_id = generate_tool_id()
                    current_tool_use_name = step["action"]["action"]

                    current_tool_use = {
                        "type": "tool_use",
                        "name": "computer",
                        "id": current_tool_use_id,
                        "input": {"action": current_tool_use_name},
                    }

                    if step["action"]["coordinate"]:
                        [x, y] = step["action"]["coordinate"]
                        norm_x = float(x) / float(step["state"]["width"])
                        norm_y = float(y) / float(step["state"]["height"])
                        current_tool_use["input"]["coordinate"] = [norm_x, norm_y]
                        # action_arg["coordinate"] = (norm_x, norm_y)

                    if step["action"]["text"]:
                        current_tool_use["input"]["text"] = step["action"]["text"]

                        # action_arg["text"] = step["action"]["text"]

                    tool_use_content.append(current_tool_use)

                    training_example.append(
                        {"role": "assistant", "content": tool_use_content}
                    )

                    last_function_name = current_tool_use_name
                    last_function_use_id = current_tool_use_id

                examples.append(training_example)

            with open(f"molmo/{filename}", "w") as jsonl_file:
                for example in examples:
                    jsonl_entry = json.dumps(example)
                    jsonl_file.write(jsonl_entry + "\n")

Processing 4c4ab02e-a617-490d-96c6-333d7fa82fe9.jsonl
Processing 627a99f1-b2e4-4ce4-93f5-89e691f8c879.jsonl
Processing 27a8bf82-2e7e-48c3-8fb1-66909c13d585.jsonl
Processing f385156c-4f2e-410f-bc73-7ec6d0f44448.jsonl
Processing 6b627cbc-a45e-4f7e-9d02-bfca1a41070a.jsonl
Processing c64dcaa1-555d-43f3-adb4-3452c4e5d09b.jsonl
Processing 10b2af14-f708-4abe-94fc-00163d11cb56.jsonl
Processing 0fb1c90e-a494-4de9-86ac-d4e5366869c4.jsonl
Processing b02e47ac-c1a6-4f5c-886f-f32af745e7f3.jsonl
Processing 82c5913d-8392-44a2-8873-6627a281fa23.jsonl
Processing d3a4e6c3-65e8-46f7-8dd7-439bc559af2a.jsonl
Processing 77be98ff-e4db-4745-9b87-6ce69754c4c2.jsonl
Processing 7c2ab1ed-54b7-4316-8b2f-d7467e01b540.jsonl
Processing 10de6ac5-a69e-4d09-87fd-5737ca6f4b99.jsonl
Processing b1eb6602-3aa7-40ee-8968-69d83e7ef21c.jsonl
Processing 86afd67c-1bff-455c-baa7-e18dcb64b0f3.jsonl
Processing 3b390b60-5c04-40ee-a5af-353def43ffbc.jsonl
Processing 1a807a1c-d3b2-425e-9684-2a9e79846676.jsonl
Processing e832e1f9-3a9d-440

In [8]:
"""Data processing utilities for Molmo training data."""


def convert_base64_to_image(base64_string: str) -> Image.Image:
    """Converts a base64 string to a PIL Image."""
    base64_string = (
        base64_string.split(",")[1] if "," in base64_string else base64_string
    )
    image_data = base64.b64decode(base64_string)
    return Image.open(BytesIO(image_data))


def yield_jsonl_records(
    mind2web_dir: Path, max_files: Optional[int] = None, max_lines: Optional[int] = None
) -> Generator:
    """Generator function that yields records from JSONL files.

    Args:
        mind2web_dir: Directory containing JSONL files
        max_files: Maximum number of files to process
        max_lines: Maximum number of lines to process per file

    Yields:
        Dict containing the parsed JSON record
    """
    jsonl_files = [f for f in mind2web_dir.glob("*.jsonl")]

    if max_files is not None:
        jsonl_files = jsonl_files[:max_files]

    print(f"Processing {len(jsonl_files)} files...")

    for file_path in tqdm(jsonl_files, desc="Processing files"):
        line_count = 0

        with open(file_path, "r") as file:
            for line in file:
                if max_lines is not None and line_count >= max_lines:
                    break

                try:
                    record = json.loads(line)
                    yield record
                    line_count += 1
                except json.JSONDecodeError as e:
                    print(f"Error in {file_path}: {e}")
                    continue


def process_record(record: List[Dict[str, Any]]) -> Dict[str, Any]:
    """Process a single JSONL record.

    Args:
        record: List of messages from the JSONL record

    Returns:
        Dict containing processed messages and images
    """
    empty_content = {"text": None}
    # empty_content = {"index": None, "text": None}

    messages = []
    images = []

    for msg in record:
        # Ensure msg["content"] is a list
        initial_content = (
            msg["content"]
            if isinstance(msg["content"], list)
            else [{"type": "text", "text": msg["content"]}]
        )

        # Append empty content K-V pairs to dictionaries in list
        empty_filled_content = [
            {**empty_content, **content_entry} for content_entry in initial_content
        ]

        # Go through each item of the list now and format
        for entry in empty_filled_content:
            if entry["type"] == "tool_result":
                for sub_entry in entry["content"]:
                    if sub_entry["type"] == "image":
                        image = convert_base64_to_image(sub_entry["source"]["data"])
                        images.append(image)
                        sub_entry["source"].pop("data")
                        sub_entry["source"]["image_index"] = len(images) - 1

            # if entry["type"] == "image_url":
            #     base64_url = entry["image_url"]["url"]
            #     image = convert_base64_to_image(base64_url)
            #     images.append(image)

            #     entry["type"] = "image"
            #     entry["image_index"] = len(images) - 1
            #     entry["text"] = None
            #     entry.pop("image_url", None)
            # elif "value" in entry:
            #     entry["text"] = entry.pop("value")

        messages.append({**msg, "content": empty_filled_content})

    return {"messages": messages, "images": images}


def create_dataset_features() -> Features:
    """Create the feature schema for the dataset."""
    return Features(
        {
            "messages": Sequence(
                {
                    "role": Value("string"),
                    "content": Sequence(
                        {
                            "index": Value("null"),
                            "text": Value("string"),
                            "type": Value("string"),
                            "image_index": Value("int64"),
                        }
                    ),
                    "tool_calls": Sequence(
                        {"name": Value("string"), "arguments": Value("string")},
                        length=-1,
                    ),
                }
            ),
            "images": Sequence(DSImage()),
        }
    )


def process_and_save_dataset(
    mind2web_dir: Path,
    save_dir: Path,
    max_files: Optional[int] = None,
    max_lines: Optional[int] = None,
    test_mode: bool = False,
) -> Dataset:
    """Process and save the dataset.

    Args:
        mind2web_dir: Input directory containing JSONL files
        save_dir: Output directory for processed dataset
        max_files: Maximum number of files to process
        max_lines: Maximum number of lines to process per file
        test_mode: Whether to save in test mode

    Returns:
        Processed Dataset
    """
    print("Starting data processing...")

    dataset = Dataset.from_generator(
        lambda: (
            process_record(record)
            for record in yield_jsonl_records(mind2web_dir, max_files, max_lines)
        )
    ).cast_column("images", Sequence(DSImage()))

    print(f"\nProcessed {len(dataset)} examples")

    # Determine save directory based on test mode
    final_save_dir = save_dir / "test" if test_mode else save_dir
    final_save_dir.mkdir(parents=True, exist_ok=True)

    dataset.save_to_disk(str(final_save_dir))
    print(f"\nDataset saved to {final_save_dir}")

    # Verify the saved dataset
    print("\nVerifying saved dataset...")
    loaded_dataset = Dataset.load_from_disk(str(final_save_dir))
    print("Dataset size:", len(loaded_dataset))
    print("\nFirst example:")
    first_example = loaded_dataset[0]
    print("Number of messages:", len(first_example["messages"]))
    print("Number of images:", len(first_example["images"]))

    return loaded_dataset

In [9]:
MIND2WEB_DIR = "/Users/macdev/Documents/repos/cerebellum/training/molmo"
SAVE_DIR = "/Users/macdev/Documents/repos/cerebellum/training/processed"

In [10]:
# Process the Molmo dataset
processed_dataset = process_and_save_dataset(
    mind2web_dir=Path(MIND2WEB_DIR),
    save_dir=Path(SAVE_DIR),
)

Starting data processing...

Processed 18989 examples


Saving the dataset (0/2 shards):   0%|          | 0/18989 [00:00<?, ? examples/s]


Dataset saved to /Users/macdev/Documents/repos/cerebellum/training/processed

Verifying saved dataset...
Dataset size: 18989

First example:
Number of messages: 5
Number of images: 1


In [11]:
# Assuming simplest conversion of Llava back to Molmo of dumping messages as string.
top_N = 5
top_N_list = []
bottom_N_list = []
for idx, item in enumerate(processed_dataset):
    this_len = len(json.dumps(item["messages"]))
    top_N_list.append((this_len, idx))
    bottom_N_list.append((this_len, idx))
    top_N_list = sorted(top_N_list, key=lambda tup: tup[0], reverse=True)[:top_N]
    bottom_N_list = sorted(bottom_N_list, key=lambda tup: tup[0], reverse=False)[:top_N]

In [12]:
for idx, item in enumerate(top_N_list):
    print(f"{idx}. Item {item[1]} has length {item[0]}")

for idx, item in enumerate(bottom_N_list):
    print(f"{idx}. Item {item[1]} has length {item[0]}")

0. Item 13175 has length 73319
1. Item 13174 has length 72631
2. Item 13173 has length 71913
3. Item 13172 has length 71217
4. Item 13171 has length 70521
0. Item 10484 has length 2815
1. Item 13523 has length 2819
2. Item 17056 has length 2825
3. Item 8714 has length 2829
4. Item 4550 has length 2832


In [13]:
# Test some basic tokenization
from transformers import AutoProcessor

# load the processor
processor = AutoProcessor.from_pretrained(
    "allenai/Molmo-7B-O-0924",
    trust_remote_code=True,
    torch_dtype="auto",
    device_map="auto",
)


for idx, item in enumerate(top_N_list):
    tokens = processor.process(text=json.dumps(processed_dataset[item[1]]["messages"]))
    print(f"{idx}. Item {item[1]} has tokenization length {len(tokens['input_ids'])}")

for idx, item in enumerate(bottom_N_list):
    tokens = processor.process(text=json.dumps(processed_dataset[item[1]]["messages"]))
    print(f"{idx}. Item {item[1]} has tokenization length {len(tokens['input_ids'])}")

Token indices sequence length is longer than the specified maximum sequence length for this model (26291 > 8192). Running this sequence through the model will result in indexing errors


0. Item 13175 has tokenization length 26292
1. Item 13174 has tokenization length 26140
2. Item 13173 has tokenization length 25809
3. Item 13172 has tokenization length 25544
4. Item 13171 has tokenization length 25367
0. Item 10484 has tokenization length 802
1. Item 13523 has tokenization length 806
2. Item 17056 has tokenization length 806
3. Item 8714 has tokenization length 805
4. Item 4550 has tokenization length 813
